In [1]:
# static data read

In [3]:
static = spark.read.json("activity-data/")

In [4]:
static.show(2)

+-------------+-------------------+--------+-----+------+----+-----+------------+------------+------------+
| Arrival_Time|      Creation_Time|  Device|Index| Model|User|   gt|           x|           y|           z|
+-------------+-------------------+--------+-----+------+----+-----+------------+------------+------------+
|1424686735090|1424686733090638193|nexus4_1|   18|nexus4|   g|stand| 3.356934E-4|-5.645752E-4|-0.018814087|
|1424686735292|1424688581345918092|nexus4_2|   66|nexus4|   g|stand|-0.005722046| 0.029083252| 0.005569458|
+-------------+-------------------+--------+-----+------+----+-----+------------+------------+------------+
only showing top 2 rows



In [5]:
# dynamic data processing
streaming = spark.readStream.schema(static.schema)\
            .option("maxFilesPerTrigger",10)\
            .json("activity-data/")\
            .groupBy("gt").count()

In [6]:
query = streaming.writeStream.outputMode("complete").queryName("tb_gt_count")\
                    .format("memory").start()

24/03/26 15:16:20 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-93c8a7e8-9b8f-4c88-ad40-7868206b4c4b. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/03/26 15:16:20 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [9]:
spark.sql("select * from tb_gt_count").show()

+----------+-------+
|        gt|  count|
+----------+-------+
|  stairsup| 836598|
|       sit| 984714|
|     stand| 910783|
|      walk|1060402|
|      bike| 863710|
|stairsdown| 749059|
|      null| 835725|
+----------+-------+

